### Installation

In [ ]:
#!pip install --no-deps unsloth vllm

In [ ]:
import sys, re, requests; modules = list(sys.modules.keys())
for x in modules: sys.modules.pop(x) if "PIL" in x or "google" in x else None
!pip install --no-deps bitsandbytes accelerate xformers==0.0.29.post3 peft "trl==0.15.2" triton cut_cross_entropy unsloth_zoo
!pip install sentencepiece protobuf datasets huggingface_hub hf_transfer

f = requests.get("https://raw.githubusercontent.com/vllm-project/vllm/refs/heads/main/requirements/common.txt").content
with open("vllm_requirements.txt", "wb") as file:
  file.write(re.sub(rb"(transformers|numpy|xformers)[^\n]{1,}\n", b"", f))
!pip install -r vllm_requirements.txt

### Unsloth

`FastModel` supports loading nearly any model now! This includes Vision and Text models!

In [ ]:
from unsloth import FastModel
import torch

model, tokenizer = FastModel.from_pretrained(
    model_name = "unsloth/gemma-3-4b-it",
    max_seq_length = 2048,
    load_in_4bit = True,
    load_in_8bit = False,
    full_finetuning = False,
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
Unsloth: Failed to patch SmolVLMForConditionalGeneration forward function.
🦥 Unsloth Zoo will now patch everything to make training faster!
INFO 04-27 14:05:54 [__init__.py:239] Automatically detected platform cuda.
==((====))==  Unsloth 2025.4.1: Fast Gemma3 patching. Transformers: 4.51.3. vLLM: 0.8.4.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: Using float16 precision for gemma3 won't work! Using float32.


model.safetensors:   0%|          | 0.00/4.56G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/210 [00:00<?, ?B/s]

processor_config.json:   0%|          | 0.00/70.0 [00:00<?, ?B/s]

chat_template.json:   0%|          | 0.00/1.61k [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/1.16M [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.69M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/33.4M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/35.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/670 [00:00<?, ?B/s]

We now add LoRA adapters so we only need to update a small amount of parameters!

In [ ]:
model = FastModel.get_peft_model(
    model,
    finetune_vision_layers     = False,
    finetune_language_layers   = True,
    finetune_attention_modules = True,
    finetune_mlp_modules       = True,

    r = 8,
    lora_alpha = 8,
    lora_dropout = 0,
    bias = "none",
    random_state = 3407,
)

Unsloth: Making `model.base_model.model.language_model.model` require gradients


In [ ]:
#!pip install qdrant_client

In [ ]:
from qdrant_client import QdrantClient

qdrant_client = QdrantClient(
    url="https://28866197-ecc2-4b02-9bb2-175b64425708.us-west-1-0.aws.cloud.qdrant.io:6333",
    api_key=API_KEY,
)

qdrant_client.recreate_collection(
    collection_name="assignment4",
    vectors_config={
        "size": 384,
        "distance": "Cosine"
    }
)

print(qdrant_client.get_collections())

<ipython-input-47-a86abd2bff5f>:8: DeprecationWarning: `recreate_collection` method is deprecated and will be removed in the future. Use `collection_exists` to check collection existence and `create_collection` instead.
  qdrant_client.recreate_collection(


collections=[CollectionDescription(name='assignment4')]


In [ ]:
#!pip install httpx beautifulsoup4 sentence-transformers scikit-learn


In [ ]:
import httpx
from bs4 import BeautifulSoup

url1 = "https://openrouter.ai/docs/features/provider-routing.md"
url2 = "https://openrouter.ai/docs/api-reference/overview.md"
url3 ="https://openrouter.ai/models"

def fetch_page_text(url):
    response = httpx.get(url)
    soup = BeautifulSoup(response.text, "html.parser")
    text = soup.get_text(separator="\n")
    return text

raw_text = fetch_page_text(url3)
print(raw_text[50])

n


In [ ]:
def chunk_text(text, max_length=500):
    paragraphs = text.split("\n")
    chunks, current = [], ""

    for para in paragraphs:
        if len(current) + len(para) < max_length:
            current += " " + para
        else:
            chunks.append(current.strip())
            current = para
    if current:
        chunks.append(current.strip())
    return [chunk for chunk in chunks if len(chunk.strip()) > 100]

chunks = chunk_text(raw_text)
print(len(chunks))

2


In [ ]:
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

embedder = SentenceTransformer("all-MiniLM-L6-v2")
chunk_embeddings = embedder.encode(chunks, convert_to_tensor=True)

In [ ]:
from qdrant_client.models import PointStruct

points = []

for idx, (embedding, chunk) in enumerate(zip(chunk_embeddings, chunks)):
    points.append(
        PointStruct(
            id=idx,
            vector=embedding.tolist(),
            payload={"text": chunk}
        )
    )

qdrant_client.upsert(
    collection_name='assignment4',
    points=points
)

print({len(points)})



{2}


 You can also use a `TextStreamer` for continuous inference - so you can see the generation token by token, instead of waiting the whole time!

In [ ]:
query = input("Input Prompt")
query_vector = embedder.encode(query).tolist()

search_result = qdrant_client.search(
    collection_name='assignment4',
    query_vector=query_vector,
    limit=5
)

context = ""

for hit in search_result:
    context += hit.payload["text"] + "\n"

messages = [{
    "role": "user",
    "content": [{"type" : "text", "text" : f"Please answer this prompt: {query} based on this context: {context}",}]
}]
text = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt = True,
)

from transformers import TextStreamer
_ = model.generate(
    **tokenizer([text], return_tensors = "pt").to("cuda"),
    max_new_tokens = 2000,
    temperature = 1.0, top_p = 0.95, top_k = 64,
    streamer = TextStreamer(tokenizer, skip_prompt = True),
)

Input PromptWhich fields determine the routing strategy for OpenRouter?


<ipython-input-57-f6dbc6750423>:4: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `query_points` instead.
  search_result = qdrant_client.search(


The routing strategy for OpenRouter is determined by several fields, primarily focused on selecting the most appropriate model and provider to fulfill a user's request. Here's a breakdown of the key factors:

1.  **`model` Parameter:**
    *   **Explicit Selection:** If the `model` parameter is provided in the request, OpenRouter will explicitly use that model.  It will select the least expensive and best GPUs available for that model.
    *   **Default Model:** If the `model` parameter is omitted, OpenRouter defaults to the user's configured default model.

2.  **Supported Models:**
    *   OpenRouter maintains a list of supported models. When a request comes in, it checks if the requested `model` is on this list.

3.  **Provider Availability and Cost:**
    *   OpenRouter uses multiple AI model providers (e.g., OpenAI, others). It dynamically chooses the provider that offers the most suitable combination of cost and GPU availability.  It aims for the least expensive and fastest optio

In [ ]:
from qdrant_client import QdrantClient

qdrant_client = QdrantClient(
    url="https://28866197-ecc2-4b02-9bb2-175b64425708.us-west-1-0.aws.cloud.qdrant.io:6333",
    api_key=API_KEY,
)

query = input("Input Prompt")
query_vector = embedder.encode(query).tolist()

search_result = qdrant_client.search(
    collection_name='assignment4',
    query_vector=query_vector,
    limit=5
)

context = ""

for hit in search_result:
    context += hit.payload["text"] + "\n"

messages = [{
    "role": "user",
    "content": [{"type" : "text", "text" : f"Please answer this prompt: {query} based on this context: {context}",}]
}]
text = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt = True,
)

from transformers import TextStreamer
_ = model.generate(
    **tokenizer([text], return_tensors = "pt").to("cuda"),
    max_new_tokens = 1500,
    temperature = 1.0, top_p = 0.95, top_k = 64,
    streamer = TextStreamer(tokenizer, skip_prompt = True),
)

Input PromptWhat are the categories for the model functionalities? Please answer in dictionary format


<ipython-input-14-299e3d9d927e>:11: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `query_points` instead.
  search_result = qdrant_client.search(


Okay, based on the provided documentation, here's a breakdown of the categories for the model functionalities, presented in a dictionary format:

```json
{
  "request_schemas": [
    {
      "type": "messages",
      "description": "A list of messages representing the conversation history. Each message has a 'role' (user, assistant, system) and 'content'.",
      "required_fields": [
        "role",
        "content"
      ],
      "subtypes": [
        {"type": "text"},
        {"type": "image_url"}
      ]
    },
    {
      "type": "prompt",
      "description": "A single string representing the user's prompt.",
      "required_fields": [
        "prompt"
      ]
    }
  ],
  "response_schemas": {
    "choices": "An array of possible completion results.",
    "usage": "Information about the tokens used (prompt tokens, completion tokens, total tokens).",
    "delta": "Only present when streaming, indicating a partial response.",
    "message": "A single message from the LLM's respons